<a href="https://colab.research.google.com/github/mybenkhadda/YanPort_Duplicate/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import libraries 

In [3]:
import pandas as pd
import numpy as np
import plotly.express as px

# Load data file

In [87]:
df = pd.read_csv("data.csv")

In [5]:
df.head()

,ID,URL,CRAWL_SOURCE,PROPERTY_TYPE,NEW_BUILD,DESCRIPTION,IMAGES,SURFACE,LAND_SURFACE,BALCONY_SURFACE,...,DEALER_NAME,DEALER_TYPE,CITY_ID,CITY,ZIP_CODE,DEPT_CODE,PUBLICATION_START_DATE,PUBLICATION_END_DATE,LAST_CRAWL_DATE,LAST_PRICE_DECREASE_DATE
0,22c05930-0eb5-11e7-b53d-bbead8ba43fe,http://www.avendrealouer.fr/location/levallois...,A_VENDRE_A_LOUER,APARTMENT,False,"Au rez de chaussée d'un bel immeuble récent,ap...","[""https://cf-medias.avendrealouer.fr/image/_87...",72.0,NaN,NaN,...,Lamirand Et Associes,AGENCY,54178039,Levallois-Perret,92300.0,92,2017-03-22T04:07:56.095,NaN,2017-04-21T18:52:35.733,NaN
1,8d092fa0-bb99-11e8-a7c9-852783b5a69d,https://www.bienici.com/annonce/ag440414-16547...,BIEN_ICI,APARTMENT,False,Je vous propose un appartement dans la rue Col...,"[""http://photos.ubiflow.net/440414/165474561/p...",48.0,NaN,NaN,...,Proprietes Privees,MANDATARY,54178039,Levallois-Perret,92300.0,92,2018-09-18T11:04:44.461,NaN,2019-06-06T10:08:10.89,2018-09-25
2,44b6a5c0-3466-11e9-8213-25cc7d9bf5fc,https://www.bellesdemeures.com/annonces/vente/...,BELLES_DEMEURES,APARTMENT,False,"Dans un cadre arboré, calme et fleuri, un pent...","[""https://v.seloger.com/s/width/965/visuels/0/...",267.0,NaN,NaN,...,Propriétés Parisiennes,AGENCY,54178039,Levallois-Perret,92300.0,92,2019-02-19T16:49:03.547,NaN,2019-06-13T08:22:14.314,NaN
3,e9e07ed0-812f-11e8-82aa-61eacebe4584,https://www.seloger.com/annonces/locations/bur...,SE_LOGER,PREMISES,False,"""Le meilleur coworking flexible de la ville, 5...","[""https://pix.yanport.com/ads/e9e07ed0-812f-11...",50.0,NaN,NaN,...,Iwg,AGENCY,54178039,Levallois-Perret,92300.0,92,2018-07-06T15:18:59.805,NaN,2019-06-18T10:40:07.405,NaN
4,872302b0-5a21-11e9-950c-510fefc1ed35,https://www.bellesdemeures.com/annonces/vente/...,BELLES_DEMEURES,HOUSE,False,Levallois - Parc de la Planchette A toute prox...,"[""https://v.seloger.com/s/width/966/visuels/0/...",330.0,NaN,NaN,...,Daniel Feau Neuilly,AGENCY,54178039,Levallois-Perret,92300.0,92,2019-04-08T17:12:20.123,NaN,2019-06-14T15:02:18.155,2019-06-14


# Explore the data

In [6]:
print(f"Shape of the dataframe is : {df.shape}")

Shape of the dataframe is : (2164, 57)


In [7]:
print(f"Columns in the dataframe are : {[i for i in df.columns]}")

Columns in the dataframe are : ['ID', 'URL', 'CRAWL_SOURCE', 'PROPERTY_TYPE', 'NEW_BUILD', 'DESCRIPTION', 'IMAGES', 'SURFACE', 'LAND_SURFACE', 'BALCONY_SURFACE', 'TERRACE_SURFACE', 'ROOM_COUNT', 'BEDROOM_COUNT', 'BATHROOM_COUNT', 'LUNCHROOM_COUNT', 'TOILET_COUNT', 'FURNISHED', 'FIREPLACE', 'AIR_CONDITIONING', 'GARDEN', 'SWIMMING_POOL', 'BALCONY', 'TERRACE', 'CELLAR', 'PARKING', 'PARKING_COUNT', 'HEATING_TYPES', 'HEATING_MODE', 'FLOOR', 'FLOOR_COUNT', 'CONSTRUCTION_YEAR', 'ELEVATOR', 'CARETAKER', 'ENERGY_CONSUMPTION', 'GREENHOUSE_GAS_CONSUMPTION', 'MARKETING_TYPE', 'PRICE', 'PRICE_M2', 'PRICE_EVENTS', 'RENTAL_EXPENSES', 'RENTAL_EXPENSES_INCLUDED', 'DEPOSIT', 'FEES', 'FEES_INCLUDED', 'EXCLUSIVE_MANDATE', 'AGENCIES_UNWANTED', 'OCCUPIED', 'DEALER_NAME', 'DEALER_TYPE', 'CITY_ID', 'CITY', 'ZIP_CODE', 'DEPT_CODE', 'PUBLICATION_START_DATE', 'PUBLICATION_END_DATE', 'LAST_CRAWL_DATE', 'LAST_PRICE_DECREASE_DATE']


In [76]:
df.isnull().sum()

ID                             0
URL                            0
CRAWL_SOURCE                   0
PROPERTY_TYPE                  0
NEW_BUILD                    191
DESCRIPTION                    4
IMAGES                         0
SURFACE                      114
TERRACE_SURFACE             2139
ROOM_COUNT                   329
BEDROOM_COUNT               1468
FURNISHED                   1697
GARDEN                         0
SWIMMING_POOL                  0
TERRACE                        0
PARKING                        0
PARKING_COUNT                  0
HEATING_TYPES                  0
HEATING_MODE                   0
FLOOR                          0
FLOOR_COUNT                 1699
CONSTRUCTION_YEAR              0
ELEVATOR                       0
CARETAKER                      0
MARKETING_TYPE                 0
PRICE                          0
PRICE_M2                     173
RENTAL_EXPENSES             1723
RENTAL_EXPENSES_INCLUDED    1564
DEPOSIT                     2109
FEES      

> Let's delete the columns with 100% NaN values

In [88]:
df = df.drop(["LAND_SURFACE","BALCONY_SURFACE", "BATHROOM_COUNT", "LUNCHROOM_COUNT", "TOILET_COUNT", "FIREPLACE", "AIR_CONDITIONING","BALCONY", "CELLAR", "ENERGY_CONSUMPTION", "GREENHOUSE_GAS_CONSUMPTION", "PUBLICATION_END_DATE"], axis=1)

## Crawl Source

In [10]:
fig = px.histogram(
    df, x = "CRAWL_SOURCE"
)
fig.show()

> This column could be deleted later to reduce complexity

## PROPERTY_TYPE

In [11]:
fig = px.histogram(
    df, x = "PROPERTY_TYPE"
)
fig.show()

## NEW_BUILD

In [12]:
fig = px.histogram(
    df, x = "NEW_BUILD"
)
fig.show()

## SURFACE

In [13]:
fig = px.box(df, y="SURFACE")
fig.show()

## ROOM_COUNT

In [14]:
fig = px.histogram(
    df, x = "ROOM_COUNT"
)
fig.show()

## BEDROOM_COUNT

In [15]:
fig = px.histogram(
    df, x = "BEDROOM_COUNT"
)
fig.show()

## FURNISHED

In [16]:
fig = px.histogram(
    df, x = "FURNISHED"
)
fig.show()

## GARDEN

> Publications that didn't mention the GARDEN means that they don't have any, let's fill it with 0

In [17]:
df['GARDEN'] = df['GARDEN'].fillna(0)

In [18]:
fig = px.histogram(
    df, x = "GARDEN"
)
fig.show()

## SWIMMING_POOL

In [19]:
df['SWIMMING_POOL'] = df['SWIMMING_POOL'].fillna(0)

In [20]:
fig = px.histogram(
    df, x = "SWIMMING_POOL"
)
fig.show()

## TERRACE

In [21]:
df['TERRACE'] = df['TERRACE'].fillna(0)

In [22]:
fig = px.histogram(
    df, x = "TERRACE"
)
fig.show()

## PARKING

In [23]:
fig = px.histogram(
    df, x = "PARKING"
)
fig.show()

## PARKING_COUNT

In [24]:
df["PARKING"].value_counts()

False    2007
True      157
Name: PARKING, dtype: int64

> Null values are basically 0,  since the column PARKING is False

In [25]:
df['PARKING_COUNT'] = df['PARKING_COUNT'].fillna(0)

In [26]:
fig = px.histogram(
    df, x = "PARKING_COUNT"
)
fig.show()

## HEATING_TYPES

In [27]:
fig = px.histogram(
    df, x = "HEATING_TYPES"
)
fig.show()

## HEATING_MODE

In [28]:
df['HEATING_MODE'] = df['HEATING_MODE'].fillna("-")

In [29]:
fig = px.histogram(
    df, x = "HEATING_MODE"
)
fig.show()

## FLOOR

In [30]:
df['FLOOR'] = df['FLOOR'].fillna(df['FLOOR'].median())

In [31]:
fig = px.histogram(
    df, x = "FLOOR"
)
fig.show()

## COUNT_FLOOR

In [32]:
fig = px.histogram(
    df, x = "FLOOR_COUNT"
)
fig.show()

## CONSTRUCTION_YEAR             

In [33]:
df['CONSTRUCTION_YEAR'] = df['CONSTRUCTION_YEAR'].fillna(df['CONSTRUCTION_YEAR'].median())

In [34]:
fig = px.histogram(
    df, x = "CONSTRUCTION_YEAR", nbins = 50
)
fig.show()

## ELEVATOR

In [35]:
df['ELEVATOR'] = df['ELEVATOR'].fillna(False)

In [36]:
fig = px.histogram(
    df, x = "ELEVATOR"
)
fig.show()

## CARETAKER

In [37]:
df['CARETAKER'] = df['CARETAKER'].fillna(False)

In [38]:
fig = px.histogram(
    df, x = "CARETAKER"
)
fig.show()

## MARKETING_TYPE

In [39]:
fig = px.histogram(
    df, x = "MARKETING_TYPE"
)
fig.show()

## PRICE

In [65]:
import re
pattern = "\"price\":([0-9]+)"
l = []
for i in range(df.shape[0]):
  l.append(int(re.findall(pattern, df[["PRICE_EVENTS"]].iloc[i][0])[0]))

In [68]:
df["PRICE"] = l

In [73]:
fig = px.box(
    df, y = "PRICE"
)
fig.show()

## PRICE_M2

## PRICE_EVENTS

In [75]:
df = df.drop(["PRICE_EVENTS"], axis = 1)

## RENTAL_EXPENSES_INCLUDED

In [85]:
df['RENTAL_EXPENSES_INCLUDED'] = df['RENTAL_EXPENSES_INCLUDED'].fillna(True)

In [86]:
fig = px.histogram(
    df, x = "RENTAL_EXPENSES_INCLUDED"
)
fig.show()

In [83]:
df.iloc[0]

ID                                       22c05930-0eb5-11e7-b53d-bbead8ba43fe
URL                         http://www.avendrealouer.fr/location/levallois...
CRAWL_SOURCE                                                 A_VENDRE_A_LOUER
PROPERTY_TYPE                                                       APARTMENT
NEW_BUILD                                                               False
DESCRIPTION                 Au rez de chaussée d'un bel immeuble récent,ap...
IMAGES                      ["https://cf-medias.avendrealouer.fr/image/_87...
SURFACE                                                                  72.0
TERRACE_SURFACE                                                           NaN
ROOM_COUNT                                                                3.0
BEDROOM_COUNT                                                             NaN
FURNISHED                                                               False
GARDEN                                                          